In [1]:
import pandas as pd
import numpy as np
import regex as re
from bs4 import BeautifulSoup
import spacy

In [2]:
data = pd.read_csv('data_2023-07-09 10_45_27 AM.csv')
data.head()

,ROW_ID,MODULE,LEVEL2,NAME,DESCRIPTION,KEYWORDS,ORACLE_MODULE,QUERY_PATH
0,1,Finance,Accounts Payable,Account Segment By Modified Date Report,The report generates COA,COA;account segment;modified date report,Oracle R13,https://pwc.sharepoint.com/sites/US-ADV-Oracle...
1,2,Finance,Accounts Payable,ACH Formats,ACH payment formats for both CCD and PPD,ACH;CCD;PPD;payment,Oracle R13,https://pwc.sharepoint.com/sites/US-ADV-Oracle...
2,3,Finance,Accounts Payable,AP Aging Summary Report,Report is developed to extract the AP aging de...,AP;date;debit balance;supplier,Oracle R13,https://pwc.sharepoint.com/sites/US-ADV-Oracle...
3,4,ERP,Other,Approval Hierarchy Report,Approval Hierarchy Report,approval;hierarchy,Oracle R13,https://pwc.sharepoint.com/sites/US-ADV-Oracle...
4,5,ERP,Other,ESS Diagnostics Dashboard,ESS Diagnostics Dashboard,ESS;diagnostics;dashboard,Oracle R13,https://pwc.sharepoint.com/sites/US-ADV-Oracle...


In [3]:
# creating a lemmatizer object
nlp = spacy.load('en_core_web_sm')
    
# converting stop words to a set for faster processing
stopwords = spacy.lang.en.stop_words.STOP_WORDS
stopwords.difference_update({'not', 'no'})
new_stopwords = set(stopwords.copy())
def clean_data(data, pos):
    # now we will interate through each review in the list and clean the data
    for i in range(data.shape[0]):
        # get the review
        review = data.iloc[i,pos]

        # remove the html tags
        clean_1_review = BeautifulSoup(review, features="html.parser").get_text()

        # convert to lower case
        clean_2_review = clean_1_review.lower()

        # remove any url's
        clean_3_review_1 = re.sub(r'http\S+', '', clean_2_review)
        clean_3_review = re.sub(r'www\S+', '', clean_3_review_1)

        # remove any non-letters
        clean_4_review = re.sub('[^a-zA-Z]', ' ', clean_3_review)
        clean_4_review = ' '.join(clean_4_review.split())

        # use spacy to tokenize the words
        clean_5_review = nlp(clean_4_review)

        # removing stopwords and lemmatizing the words
        clean_6_review = [word.lemma_ for word in clean_5_review if word not in new_stopwords]

        # join the words back into one string
        clean_7_review = ' '.join(clean_6_review)

        # update the review list with the cleaned review
        data.iloc[i,pos] = clean_7_review
    
def clean_custom_data(review):
    # remove the html tags
    clean_1_review = BeautifulSoup(review, features="html.parser").get_text()
    
    # convert to lower case
    clean_2_review = clean_1_review.lower()
    
    # remove any url's
    clean_3_review_1 = re.sub(r'http\S+', '', clean_2_review)
    clean_3_review = re.sub(r'www\S+', '', clean_3_review_1)
    
    # remove any non-letters
    clean_4_review = re.sub('[^a-zA-Z]', ' ', clean_3_review)
    clean_4_review = ' '.join(clean_4_review.split())
    
    # use spacy to tokenize the words
    clean_5_review = nlp(clean_4_review)
    
    # removing stopwords and lemmatizing the words
    clean_6_review = [word.lemma_ for word in clean_5_review if word not in new_stopwords]
    
    # join the words back into one string
    clean_7_review = ' '.join(clean_6_review)

    return clean_7_review

In [4]:
clean_data(data,4)

In [5]:
data.head()

,ROW_ID,MODULE,LEVEL2,NAME,DESCRIPTION,KEYWORDS,ORACLE_MODULE,QUERY_PATH
0,1,Finance,Accounts Payable,Account Segment By Modified Date Report,the report generate coa,COA;account segment;modified date report,Oracle R13,https://pwc.sharepoint.com/sites/US-ADV-Oracle...
1,2,Finance,Accounts Payable,ACH Formats,ach payment format for both ccd and ppd,ACH;CCD;PPD;payment,Oracle R13,https://pwc.sharepoint.com/sites/US-ADV-Oracle...
2,3,Finance,Accounts Payable,AP Aging Summary Report,report be develop to extract the ap age detail...,AP;date;debit balance;supplier,Oracle R13,https://pwc.sharepoint.com/sites/US-ADV-Oracle...
3,4,ERP,Other,Approval Hierarchy Report,approval hierarchy report,approval;hierarchy,Oracle R13,https://pwc.sharepoint.com/sites/US-ADV-Oracle...
4,5,ERP,Other,ESS Diagnostics Dashboard,ess diagnostic dashboard,ESS;diagnostics;dashboard,Oracle R13,https://pwc.sharepoint.com/sites/US-ADV-Oracle...


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
transformed_output = tfidf.fit_transform(data['DESCRIPTION'])

In [7]:
data.iloc[:2,4]

0                    the report generate coa
1    ach payment format for both ccd and ppd
Name: DESCRIPTION, dtype: object

In [9]:
transformed_output_1 = pd.DataFrame(transformed_output.toarray(),columns = tfidf.get_feature_names_out())
transformed_output_1.head()

,ach,age,an,and,ap,approval,as,balance,be,both,...,ppd,project,provide,report,summary,supplier,the,this,to,warehouse
0,0.000000,0.00000,0.0,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,...,0.000000,0.0,0.0,0.352578,0.00000,0.00000,0.411446,0.0,0.000000,0.0
1,0.368985,0.00000,0.0,0.302573,0.00000,0.000000,0.00000,0.00000,0.00000,0.368985,...,0.368985,0.0,0.0,0.000000,0.00000,0.00000,0.000000,0.0,0.000000,0.0
2,0.000000,0.38458,0.0,0.000000,0.19229,0.000000,0.19229,0.19229,0.19229,0.000000,...,0.000000,0.0,0.0,0.114078,0.19229,0.19229,0.399374,0.0,0.315361,0.0
3,0.000000,0.00000,0.0,0.000000,0.00000,0.652057,0.00000,0.00000,0.00000,0.000000,...,0.000000,0.0,0.0,0.386839,0.00000,0.00000,0.000000,0.0,0.000000,0.0
4,0.000000,0.00000,0.0,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,0.000000,...,0.000000,0.0,0.0,0.000000,0.00000,0.00000,0.000000,0.0,0.000000,0.0


In [10]:
transformed_output_1.shape

(6, 45)

In [13]:
data_2 = pd.concat([data['ROW_ID'],transformed_output_1],ignore_index = True,axis=1)
data_2.head()

,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
0,1,0.000000,0.00000,0.0,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,...,0.000000,0.0,0.0,0.352578,0.00000,0.00000,0.411446,0.0,0.000000,0.0
1,2,0.368985,0.00000,0.0,0.302573,0.00000,0.000000,0.00000,0.00000,0.00000,...,0.368985,0.0,0.0,0.000000,0.00000,0.00000,0.000000,0.0,0.000000,0.0
2,3,0.000000,0.38458,0.0,0.000000,0.19229,0.000000,0.19229,0.19229,0.19229,...,0.000000,0.0,0.0,0.114078,0.19229,0.19229,0.399374,0.0,0.315361,0.0
3,4,0.000000,0.00000,0.0,0.000000,0.00000,0.652057,0.00000,0.00000,0.00000,...,0.000000,0.0,0.0,0.386839,0.00000,0.00000,0.000000,0.0,0.000000,0.0
4,5,0.000000,0.00000,0.0,0.000000,0.00000,0.000000,0.00000,0.00000,0.00000,...,0.000000,0.0,0.0,0.000000,0.00000,0.00000,0.000000,0.0,0.000000,0.0
